In [14]:
from pickletools import uint8
from turtle import color
import numpy as np
import matplotlib.pyplot as plt
from MNISTManager import MNISTManager
import os
import sys

> _Import each matrices in a dictionnary like this :_
* trainingImages : 60000 training 28*28 images
* trainingLabels : 60000 labels for each training images
* testingImages : 10000 testing 28*28 images
* testingLabels : 10000 labels for each testing images

In [25]:
matrices = MNISTManager(
    os.path.dirname("../datasets/train-images-idx3-ubyte.gz"),
    os.path.dirname("/../datasets/train-labels-idx1-ubyte.gz"),
    os.path.dirname("/../datasets/t10k-images-idx3-ubyte.gz"),
    os.path.dirname("/../datasets/t10k-labels-idx1-ubyte.gz"),
).getMatrices()
print(matrices)

IsADirectoryError: [Errno 21] Is a directory: '../datasets'

> _Define utilitaries functions_

In [19]:
print(matrices)
def getEuclidianDistance(x, y): return np.sqrt(np.sum(x - y)**2) #get the euclidian distance between 2 matrices

def getKNearestNeighbors(k, sortedDistances): #given a value of k and a sorteed distances list, returns the k neirest neighbors in an new list
    kDistances = sortedDistances[:k]
    return [x[1] for x in kDistances]

def getAccuraryPercentage(labels): #get the acccuracy of the given labels list clompared to the training labels
    accuracy = 0;
    for i in range(len(labels)):
        if matrices['trainingLabels'][i] == labels[i]:
            accuracy += 1
    return (accuracy / len(labels) * 100)

>_KNN algorithm to get the best value of K_

In [20]:
kValues = [(3, 0),(5, 0), (7, 0), (9, 0)]  #[(kNeigbors, accuracyPercentage)]
batchSize = 600

print(matrices)

for kIndex in range(len(kValues)):
    correspondingLabels = [] # for each image, we store is found label
    for i in range(len(matrices['trainingImages'][:batchSize])):
        print(f"\t{ round(   ((kIndex / len(kValues) * 100) + ( (i / batchSize) ) / len(kValues) * 100 ), 5   ) }%", end='\r')
        sys.stdout.flush()
        distances = []
        for j in range(len(matrices['trainingImages'][:batchSize])):
            distances.append( ( getEuclidianDistance(matrices['trainingImages'][i], matrices['trainingImages'][j]), matrices['trainingLabels'][j]) )
        distances.sort(key = lambda x : x[0])
        kNearestNeighbors = getKNearestNeighbors(kValues[kIndex][0], distances)
        correspondingLabels.append( max( kNearestNeighbors, key=kNearestNeighbors.count ) ) #for each image, we add the most propable neigbor for the current value of k
    kValues[kIndex] = (kValues[kIndex][0], getAccuraryPercentage(correspondingLabels))


NameError: name 'matrices' is not defined